# Mustererkennung/Machine Learning - Assignment 6



In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from collections import defaultdict

Load the spam dataset:

In [2]:
data = np.array(pd.read_csv('/Users/Eva/Downloads/spambase.data', header=None))

X = data[:,:-1] # features
y = data[:,-1] # Last column is label

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True, stratify=y)


## Exercise 1

Implement a classification tree using python (incl. Numpy etc.) and use it on the SPAM-Dataset. Use a metric of your choice as a loss function.

(Implementation inspired by:
https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/ )


In [3]:
X.shape

(4601, 57)

In [4]:
X_train.shape, X_test.shape

((3450, 57), (1151, 57))

In [5]:
y_train.shape, y_test.shape

((3450,), (1151,))

Possible modifications/improvements:

- Another possible stopping criterion: until all leaves are pure
- Pruning to correct the depth that was added due to balance criterion: in case a node was split into a very unbalanced distribution where one child node contains no points and the other child contains all, then this split could be removed in postprocessing and the parent node can be made leaf
- With my code it can happen that two (or more?) subsequent node levels test the same variable and value and assign the same labels to both sides...either bug fix or prune
- Does the gini index need to be additionally weighted by group proportions?
- Training on the complete data takes around 2 minutes

In [1]:
class ClassificationTree():


    def __init__(self, node_min_size=2, tree_max_depth=None):
        """
        If no maximum depth is defined, the nodes are split while they contain at least node_min_size many points.
        If maximum depth is defined, nodes are split further until they contain less than node_min_size many points or
        have reached the maximum number of allowed split levels.
        """
        self.depth = 1
        self.node_min_size = node_min_size
        self.tree_max_depth = tree_max_depth
        
    ################ Tree building ################
    
    def split_data(self, X, j, v):
        """
        Split data X at variable with index j and splitting point v.
        """
        split_left = X[np.where(X[:,j]<=v)]
        split_right = X[np.where(X[:,j]>v)]
        return split_left, split_right

    
    def gini_index(self, groups):
        """
        Calculate gini index for a data set splitted on a certain variable and splitting point.
        groups: two subsets that result from splitting a data set
        y: associated class labels
        """
        gini = 0
        n = len(groups[0])+len(groups[1])
        # get class proportions for each split group
        for group in groups:
            size = float(len(group))
            y = group[:,-1]
            if size == 0:
                continue    
            score = 0
            # sum class proportions within one split groups as sum(p_mk*(1-p_mk))
            for c in set(y):
                proportions = np.count_nonzero(y==c)/size
                score += proportions*(1-proportions)
            # sum "inversely squared" class proportions per split group in weighted sum (weighted by proportion of group size)
            gini += score * size/n 
        return gini


    def split_node(self, data):
        """
        Find optimal split for a given node "node" and class labels "y" that minimizes the loss function (gini index).
        """
        split_var, split_point, split_gini, groups = 9999, 9999, 9999, None
        # for every candidate split X*p* calculate gini index
        for j in range(data.shape[1]-1):
            candidate_p = np.unique(X[:,j])
            for v in candidate_p:
                split_groups = self.split_data(data, j, v)
                s = self.gini_index(split_groups) 
                if s<split_gini: 
                    split_var, split_point, split_gini, groups = j, v, s, split_groups
        return {'split_var': split_var, 'split_point': split_point, 'split_gini': split_gini, 'groups': groups}

    
    def make_leaf(self, node_labels):
        """
        Convert node "node" to leaf by assigning it the majority vote of labels "y".
        """
        leaf_labels = list(node_labels)
        return max(set(leaf_labels), key=leaf_labels.count)

    
    def grow(self, node):
        """
        Recusively split nodes until stopping criteria are fulfilled,
        i.e. a node has too little data points in its region to be splitted further or the the has
        reached its maximum allowd depth.
        """
        left, right = node['groups']
        del(node['groups'])
        # Check for tree balance
        if not left.any() or not right.any():
            node['left'] = node['right'] = self.make_leaf(np.r_[left[:,-1], right[:,-1]])
            return None
        # Check for max depth
        if self.tree_max_depth:
            if self.depth >= self.tree_max_depth:
                node['left'] = self.make_leaf(left[:,-1])
                node['right'] = self.make_leaf(right[:,-1])
                return None
        # Check left node size
        if len(left) <= self.node_min_size-1:
            node['left'] = self.make_leaf(left[:,-1])
        else:
            node['left'] = self.split_node(left)
            self.depth += 1
            self.grow(node['left'])
        # Check right node size
        if len(right) <= self.node_min_size:
            node['right'] = self.make_leaf(right[:,-1])
        else:
            node['right'] = self.split_node(right)
            self.depth += 1
            self.grow(node['right'])
        return None

    def grow_tree(self, X, y):
        """
        Grow a decision tree for feature data X with class labels y that has at most tree_max_depth many levels
        and at least node_mins_size many data points that support each leaf prediction.
        """
        if X.shape[0] <= self.node_min_size:
            raise Exception("Data set is too small to build a tree!")
        root = self.split_node(np.c_[X,y])
        self.grow(root)
        return root
    
    ################ Classification ################
    
    def get_label(self, test, subtree):
        if type(subtree)==dict:
            if test[subtree['split_var']] <= subtree['split_point']:
                subtree = subtree['left']
                return decide(test, subtree)
            else:
                subtree = subtree['right']
                return decide(test, subtree)
        else:
            return subtree
        
        
    def predict(self, tree, X):
        y_pred = []
        for item in X:
            y_pred.append(self.get_label(item, tree))
        return y_pred
    
    ################ Evaluation ################
    
    def calculate_accuracy(self, true_y, pred_y, classes):
        """
        Calculate accuracy for a classified set.
        """
        class_sum = 0
        for class_num in classes:
            val_sum = 0
            for true_val, pred_val in zip(true_y, pred_y): 
                if class_num == true_val:
                    if true_val == pred_val:
                        val_sum += 1
            class_sum += val_sum
        return class_sum / len(true_y)

In [113]:
CT = ClassificationTree()

In [114]:
tree = CT.grow_tree(X_train, y_train)

In [115]:
y_pred = CT.predict(tree, X_test)

In [116]:
acc = 100*CT.calculate_accuracy(y_test, y_pred, set(y_test))

In [123]:
print("The trained classification tree yields an accuracy of {}% on the test set".format(round(acc,2)))

The trained classification tree yields an accuracy of 91.57% on the test set


**Exercise 1.a**

Assume that classifying a genuine E-Mail as spam is ten times worse than classifying spam as genuine. How yould you change the design of your decision tree?


(Eva): 
<br>Should we only answer or also implement here? My idea would be to use the misclassification error as loss function and to implement this penalty as a weighting scheme. <br> E.g. when calculating the misclassification error for a split, one node would have the maximum p_m,k for k=0 (real mail) and the other node would have maximum p_m,k for k=1 (spam). Then, the misclassification error for the p_m,0-node could be weighted with w_fp (weight for false positives) and the p_m,1.node could be weighted with w_fn (false negative weight) where w_fn = 10 * w_fp.
Of course a similar weighting could be implemented into the gini scoring function.

**Exercise 1.b**

Use your tree to analyze feature importance. Plot the difference between the top 5 features (check spambase.names to check what features those belong to).